# Laboratoria 10 - inne istotne zagadnienia NLP

Dzisiejsze laboratoria kończą cykl ocenianych zadań. Jest to też dobra okazja do przećwiczenia problemów, które co prawda nie były poruszane na wykładzie, ale są często spotykane w pracy.

Dzisiaj skupimy się na trzech aspektach przetwarzania języka:
<ol>
    <li>Automatycznej sumaryzacji poprzez identyfikację zdań kluczowych.</li>
    <li>Ekstrakcji relacji z tekstu nieustrukturyzowanego.</li>
    <li>Zagadnieniu Topic Modelling</li>
</ol>

Ponieważ będziemy intensywnie korzystać z biblioteki SpaCy - załadujmy model poprzez uruchomienie kodu poniżej tak, aby nie trzeba było go ładować w kolejnych komórkach.

In [1]:
!python -m spacy download en_core_web_md

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


In [2]:
import en_core_web_md
import spacy
import nltk

nlp = en_core_web_md.load()
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Automatyczna sumaryzacja

Obecnie, ilość informacji znajdujących się w sieci jest tak przytłaczająca, że nie jesteśmy w stanie czytać tyle ile chcielibyśmy. Dlatego też zaczęły rozwijać się metody automatycznej sumaryzacji tekstu, które z długich dokumentów "wyciągną" najistotniejsze fragmenty.

Istnieją przeróżne podejścia do tego problemu, kilka z nich wymieniono poniżej
<ul>
    <li>Wybierz pierwszy paragraf - artykuły często zawierają wszystkie najważniejsze informacje już we wprowadzeniu! (szczególnie te w czasopismach)</li>
    <li>Wybierz najistotniejsze zdania, które zawierają najwięcej informacji i tylko je przedstaw użytkownikowi</li>
    <li>Staraj się zrozumieć tekst, zamodeluj wiedzę z tekstu i wygeneruj własne zdania. </li>
</ul>
Na dzisiejszych laboratoriach skupimy się na drugiej z metod - Podzielimy dokument na zdania, a następnie stworzymy ranking istotności zdań, z którego wybierzemy kilka pierwszych (najlepiej ocenionych) elementów. Pierwsza jest zbyt prosta do implementacji na laboratoriach, trzecia - zbyt skomplikowana.

Aby stworzyć ranking zdań, można przyjąć następującą strategię:
<ol>
    <li>Przeprowadź preprocessing tekstu: spraw, aby tekst nie używał wielkich liter.</li>
    <li>Podziel tekst na zdania, a następnie każde z tych zdań na słowa.</li>
    <li>Ze zbioru wszystkich zdań - stwórz słownik, który każdemu słowu z tekstu przypisze liczbę wystąpień tego słowa w całym tekście (nie tylko w pojedynczym zdaniu!) (słownik powinien być w zmiennej freq). Słowo powinno znaleźć się w słowniku jeśli nie należy do zbioru stopwords (najczęstsze słowa typu: and, or, a, an) i jeśli nie jest znakiem interpunkcyjnym.</li>
    <li>Korzystając ze zbudowanego w poprzednim kroku słownika - nadaj każdemu zdaniu wartość oznaczającą jego informatywność. Informatywność zdania może być obliczona jako suma częstości odczytanych ze słownika z poprzedniego kroku (jeśli słowo występuje w słowniku, gdyż słownik nie zawiera stopwordsów!).</li>
    <li>Mając stworzony ranking - wybierz top N elementów i przedstaw je jako podsumowanie.</li>
</ol>

Przesłanka do tego podejścia jest taka, że jeśli dane słowo (które nie należy do stopwords) występuje często - jest pewnie istotne. Jeśli w zdaniu występuje dużo istotnych słów - zdanie jest lepsze z punktu widzenia sumaryzacji. Nie normalizujemy wyników długością zdania, ponieważ można przypuszczać, że dłuższe zdania będą lepszym wyborem.

**Zadanie1 (2.5 pkt)** Uzupełnij kod automatycznej sumaryzacji:
<ol>
    <li>Uzupełnij funkcję **compute_frequencies**, która dla stokenizowanych zdań (lista list) wygeneruje slownik, ktory zwróci mapowanie słowo -> liczność wystąpień tego słowa w zbiorze dokumentów. Tokeny, które są stopwordsami lub znakami interpunkcyjnymi nie powinny być dodawane do słownika.</li>
    <li>Zamień tekst na tekst pisany małymi literami, podziel go na zdania, a każde z tych zdań na słowa (pierwsze 3 linijki funkcji summarize)</li>
    <li>Uzupełnij funkcję **create_sentence_ranking**, która na wejściu otrzymuje listę stokenizowanych zdań i słownik wygenerowany przez **create_frequencies**, a na wyjściu wygeneruje słownik mapujący numer porządkowy zdania na wartość istotności tego zdania (suma częstości tokenów tego zdania pobrana z freq)</li>
</ol>

In [7]:
from nltk import word_tokenize, sent_tokenize
from collections import defaultdict, Counter
from nltk.corpus import stopwords
from string import punctuation
from pprint import pprint

stopwords = set(stopwords.words('english') + list(punctuation)) # stwórz listę tokenów, które powinny być ignorowane

def compute_frequencies(word_sent):
  # trudno sie czyta, bo python nie ma flatmapa w biliotece standardowej :/
  return Counter([x for xs in word_sent for x in xs if x not in stopwords])

def sentence_ranking(sentences, freq):
  return [sum([freq[word] for word in sent]) for sent in sentences]

def summarize(text, sentence_count):
  sents_raw = sent_tokenize(text.lower())
  sents = [word_tokenize(x) for x in sents_raw] # TODO: podziel zdania na słowa tworząc listę list (lista zdań, z których każdy element to lista tokenów w zdaniu)
  freqs = compute_frequencies(sents) # tutaj otrzymamy słownik, jeśli chcesz - wyświetl go - czy rzeczywiście częste słowa są tymi istotnymi?
  ranking = sentence_ranking(sents, freqs)
  ranked = sorted(zip(ranking, sents_raw), key=lambda x: -x[0])
  return ranked[:sentence_count]
    
text = '''
Washington (CNN) As preparations are underway for a US-North Korea summit, US officials are trying to solve the logistical issue of who will pay for North Korean leader Kim Jong Un's housing, according to a new report.

A week after abruptly scrapping the summit with Kim, President Donald Trump announced Friday that the historic talks were back on for June 12 in Singapore.
With its economy weakened from tough sanctions, Pyongyang is requiring that another country pay for Kim and his delegation's hotel bill, The Washington Post reported Friday.
According to the Post, Kim is demanding to stay at the luxury, five-star Fullerton hotel, where a presidential suite costs more than $6,000 a night.
America should be more at ease than this
America should be more at ease than this
White House and State Department officials declined to comment to the Post on the advance team planning details.
Citing two people familiar with the talks, the Post reported that the US is open to covering the costs, but is considering asking the host country, Singapore, to foot the bill.
The International Campaign to Abolish Nuclear Weapons also offered to pay for Kim's lodging with the cash received as part of its Nobel Peace Prize ($1.1 million) it won last year "in order to support peace in the Korean Peninsula and a nuclear-weapon-free world."
"Our movement is committed to the abolition of nuclear weapons and we recognize that this historic summit is a once in a generation opportunity to work for peace and nuclear disarmament," ICAN International Steering Group member Akira Kawasaki said in a statement.
The Post is also reporting that the US is expected to request a waiver of sanctions from the United Nations and US Treasury Department for expenses associated with North Korea's travel.
Trump is expected to stay at another five-star hotel, the Shangri-La, which has hosted high security events before, according to the Post.
Determining who will pay Kim's hotel bill is one of many logistical issues still being hammered out ahead of the summit, including the aircraft Kim will use to fly to Singapore and the venue where Trump and Kim will meet, the Post reported.
The relatively secluded Capella hotel on the island of Sentosa is being considered for the site of the summit, people familiar with the talks told the Post.
'''

for score, sent in summarize(text, 2): # wybierz 2 najlepsze zdania
    print('*', sent)

* determining who will pay kim's hotel bill is one of many logistical issues still being hammered out ahead of the summit, including the aircraft kim will use to fly to singapore and the venue where trump and kim will meet, the post reported.
* the international campaign to abolish nuclear weapons also offered to pay for kim's lodging with the cash received as part of its nobel peace prize ($1.1 million) it won last year "in order to support peace in the korean peninsula and a nuclear-weapon-free world."


## Ekstrakcja relacji

Zrozumienie tekstu wymaga zarówno rozumienia poszczególnych słów jak i relacji pomiędzy tymi słowami. O ile o znaczeniu pojedynczych słów mówiliśmy już trochę (embeddingi i ocena podobieństwa z ich zastosowaniem, a także problem POS-taggingu, który odkrywa jaką część mowy reprezentuje dane słowo), o tyle o relacjach między słowami  mówiliśmy niewiele. 

Relacjami między wyrazami w zdaniu rządzi gramatyka, dzięki której możemy zrozumieć jak wymienione w zdaniach idee łączą się ze sobą. Dotychczasowe badania w dziedzinie przetwarzania języka naturalnego zaproponowały tzw. drzewa zależnościowe (dependency tree lub dependency parse tree), jako wizualizację zależności gramatycznych między wyrazami w postaci drzewa. Korzeniem tego drzewa jest najważniejszy w zdaniu czasownik. Połączenia między węzłami w drzewie zależnościowym są etykietowane nazwami relacji między słowami.

Wizualizacje generowanych drzew zależnościowych dla zadanaych zdań wygenerować można pod adresem: https://explosion.ai/demos/displacy

Etykiety znajdujące się na krawędziach drzewa opisane są pod adresem: https://nlp.stanford.edu/software/dependencies_manual.pdf w rozdziale 2.

Wizualizcję drzewa zależnościowego (bez etykiet na połączeniach węzłów) możemy uzyskać użyciem SpaCy i NLTK. Uruchom poniższy kod, aby zaobserwować rezultat:

In [4]:
from nltk import Tree # przydatne do wyświetlania drzewa

doc = nlp("The quick brown fox jumps over the lazy dog. Mary met Mike.") # przykładowe zdania do przetworzenia

def to_nltk_tree(node): # tworzymy drzewo
    if node.n_lefts + node.n_rights > 0:
        return Tree(node.text, [to_nltk_tree(child) for child in node.children])
    else:
        return node.text

for sent in doc.sents:
    print(sent)
    print("-----------------------------------")
    to_nltk_tree(sent.root).pretty_print() # stwórz drzewo i pięknie je przedstaw
    print("\n\n\n")

The quick brown fox jumps over the lazy dog.
-----------------------------------
        jumps                    
  ________|______________         
 |        |             over     
 |        |              |        
 |       fox            dog      
 |    ____|_____      ___|____    
 .  The quick brown the      lazy





Mary met Mike.
-----------------------------------
     met     
  ____|____   
Mary Mike  . 







Do czego może się przydać drzewo zależnościowe? 
Możemy wykorzystać takie drzewo np. do upraszczania zdań, odkrywania relacji między elementami zdania, czy np. do odkrywania do jakiego fragmentu tekstu odnosi się fraza nacechowana emocjonalnie ("Bardzo lubię te babcine, wiejskie pierogi, ale dobrym kebabem w sumie też nie pogardzę" => lubię--pierogi, nie-pogardzę--kebabem)

Wykorzystajmy drzewo zależnościowe, aby stworzyć uproszczoną reprezentację zdania, zawierającą relację (czasownik) i argumenty tej relacji w formie relacja(argument1, argument2,...)

**Zadanie 2: Prosta ekstrakcja relacji z wykorzystaniem drzewa zależnościowego**

**Zadanie 2a (0.5 pkt)**: Wykorzystując atrybuty stworzonych przez spacy tokenów po uruchomieniu funkcji nlp() (https://spacy.io/api/token#attributes) - stwórz reprezentację CONLL, w której znajdą się następujące atrybuty (kolumny):
<ol>
<li>identyfikator słowa w dokumencie</li>
<li>tekst słowa</li>
<li>etykieta z drzewa zależnościowego na połączeniu z "rodzicem"</li>
<li>tekst rodzica z drzewa zależnościowego</li>
<li>listę dzieci z drzewa zależnościowego</li>
</ol>

Oczekiwany rezultat:

<pre>
0 The det fox []
1 quick amod fox []
2 brown amod fox []
3 fox nsubj jumps [The, quick, brown]
4 jumps ROOT jumps [fox, over, .]
5 over prep jumps [dog]
6 the det dog []
7 lazy amod dog []
8 dog pobj over [the, lazy]
9 . punct jumps []


10 Mary nsubj met []
11 met ROOT met [Mary, Mike, .]
12 Mike dobj met []
13 . punct met []
</pre>

In [5]:
from nltk import Tree # przydatne do wyświetlania drzewa

doc = nlp("The quick brown fox jumps over the lazy dog. Mary met Mike.") # przykładowe zdania do przetworzenia

i = 0
for sent in doc.sents:
  for word in sent:
    print(i, word, word.dep_, word.head, list(word.children));
    i += 1
  print()

0 The det fox []
1 quick amod fox []
2 brown amod fox []
3 fox nsubj jumps [The, quick, brown]
4 jumps ROOT jumps [fox, over, .]
5 over prep jumps [dog]
6 the det dog []
7 lazy amod dog []
8 dog pobj over [the, lazy]
9 . punct jumps []

10 Mary nsubj met []
11 met ROOT met [Mary, Mike, .]
12 Mike dobj met []
13 . punct met []



Widzimy, że najistotniejszym czasownikiem jest słowo "jumps" (korzeń drzewa zależnościowego (ROOT))
Widzimy też, że słowa grupują się odpowiednio. Dzieci słowa 'fox' to ['The', 'quick', 'brown'] - a więc określenia definiujące jaki ten lis jest! (Podobnie dla słowa dog)


**Zadanie 2b (2 pkt)** Ekstrakcja relacji.

Wiedząc jak należy pobierać informacje o drzewie zależnościowym z obiektów typu Token w SpaCy, napisz funkcję parsującą, która dla każdego zdania (zdania przetworzonego przez SpaCy) wyekstrahuje najważniejszą relację (czasownik będący ROOTem), a także argumenty tej relacji (podmiot i dopełnienie) bazując na wygenerowanym drzewie zależnościowym.

<ol>
<li>Relacja powinna zostać zapisana w zmiennej predicate</li>
<li>Podmiot, zdefiniujmy jako token ze zdania, który połączony jest z ROOTem relacją 'nsubj', zapisany powinien być w zmiennej subj.</li>
<li>orzeczenie zaś określone może być np. jako:element połączony z ROOTem relacją 'dobj', bądź, jeśli ROOT nie ma połączenia 'dobj', a połączony jest z elementem relacją 'prep' (przyimek w relacji do czasownika), to orzeczeniem jest token, który połączony jest z tym przyimkiem relacją 'pobj'. Jeśli występuje sytuacja druga, tzn. przyimek jest połączony bezpośrednio z ROOTem - a dopiero ten przyimek z określeniem, przyimek powinien zostać doklejony do napisu zapisanego w zmiennej predicate (Dla uproszczenia załóżmy, że przyimek występuje zawsze po czasowniku). Dopełnienie zapisz w zmiennej 'obj'.</li>
</ol>
Aby zrozumieć działanie dopełnienia - spójrz na oczekiwany rezultat tego zadania i na drzewo zależnościowe wygenerowane w pierwszym fragmencie kodu tej sekcji.

Oczekiwany rezultat:

<pre>
jumps over(fox, dog)
met(Mary, Mike)
</pre>

O ile drugi przykład met(Mary, Mike) jest oczywisty, to pierwszy powinien zidentyfikować słowo 'jumps' jako relację, zauważyć, że nie istnieje bezpośrednie dopełnienie (brak 'dobj' dla roota), za to mamy przyimek over, który to z kolei jest połączony z oczekiwanym dopełnieniem ('dog'). Zatem przyimek doklejamy do nazwy relacji, zamieniając dotychczasowe jumps na jumps over, a dopełnieniem staje się element połączony z przyimkiem relacją 'pobj': dog. 

In [13]:
from nltk import Tree # przydatne do wyświetlania drzewa
from spacy.symbols import VERB, prep, nsubj, pobj, dobj

doc = nlp("The quick brown fox jumps over the lazy dog. Mary met Mike.") # przykładowe zdania do przetworzenia

def parse(sent):
  pred = [sent.root.text]
  subj = None
  obj = None
  
  for word in sent:
    if word.head.text in pred:
      if word.dep == prep:
        pred.append(word.text)
      if word.dep == nsubj:
        subj = word.text
      if word.dep in [pobj, dobj]:
        obj = word.text

  pred = ' '.join(pred)
  print(f'{pred}({subj}, {obj})')
    
for sent in doc.sents:
  parse(sent)

jumps over(fox, dog)
met(Mary, Mike)


## Podsumowanie

W ramach laboratoriów dotknęliśmy wielu różnych problemów NLP:
<ol>
    <li>Lab1: Szybkie wyszukiwanie/operowanie na tekście -- wyrażenia regularne</li>
    <li>Lab2: Klasyfikacja (Tworzenie reprezentacji BagOfWords, Tokenizacja, Lematyzacja, Stemming, normalizacja TF-IDF, SVM, Naiwny Bayes)</li>
    <li>Lab3: NGramy (Reprezentacja NGram vs BagOfWords, detekcja języka, generowanie tekstu)</li>
    <li>Lab4: Embeddingi jako niskowymiarowa alternatywa dla BagOfWords/NGram (podobieństwo w przestrzeni embeddingów, embeddingi do klasyfikacji) + Poprawianie literówek (odległość Levensteina)</li>
    <li>Lab5: Sieci neuronowe (Przypomnienie z SI, sieci jako sekwencja operacji na macierzach, sieć implementowana bez użycia frameworków)</li>
    <li>Lab6: Tworzenie zasobów (Crawling/Scraping danych z sieci, tworzenie zasobów z użyciem reprezentacji CONLL)</li>
    <li>Lab7: Sieci rekurencyjne w przetwarzaniu tekstu (RNN od podstaw bez użycia frameworków, idea historii w RNN)</li>
    <li>Lab8: Detekcja sentymentu z użyciem zaawansowanych architektur sieci (GRU/LSTM/CNN)</li>
    <li>Lab9: Wykrywanie encji nazwanych i fraz rzeczownikowych</li>
    <li>Lab10: Sumaryzacja poprzez wyszukiwanie zdań kluczowych (Key-sentence extraction), modelowanie tematów (topic modelling - LDA), Ekstrakcja informacji (Drzewo zależnościowe, rozbiór gramatyczny zdań). </li>
</ol>

Użyliśmy także szeroko używanych narzędzi takich jak:
<ol>
    <li>NLTK - natural language toolkit - narzędzia do popularnych zadań NLP</li>
    <li>SpaCy - narzędzia do popularnych zadań NLP, młodsza alternatywa NLTK</li>
    <li>sklearn - narzędzie implementujące algorytmy uczenia maszynowego</li>
    <li>Keras - wysokopoziomowy framework do sztucznych sieci neuronowych</li>
    <li>gensim - narzędzie do modelowania topików</li>
    <li>Pandas - narzędzie do łatwego operowania na danych</li>
</ol>


**Ponieważ chcielibyśmy, aby ten przedmiot stawał się coraz lepszy przygotwałem krótką ankietę dotyczącą przedmiotu, będę wdzięczny za udzielenie w niej odpowiedzi! https://goo.gl/forms/jWnSAwHkzKKs86QV2 **